In [1]:
import sys
sys.path.append("PointNet_Pointnet2")  # adjust if needed

from models.pointnet2_sem_seg import get_model
import torch
import torch.nn as nn

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Instantiate model with correct num_classes and input channels
model = get_model(num_classes=2).to(device)

# ✅ Load trained weights
model.load_state_dict(torch.load("pointnet2_screw_segmentation_final.pth", map_location=device))
model.eval()


get_model(
  (sa1): PointNetSetAbstraction(
    (mlp_convs): ModuleList(
      (0): Conv2d(9, 32, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
      (2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (mlp_bns): ModuleList(
      (0-1): 2 x BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (sa2): PointNetSetAbstraction(
    (mlp_convs): ModuleList(
      (0): Conv2d(67, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    )
    (mlp_bns): ModuleList(
      (0-1): 2 x BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (sa3): PointNetSetAbstraction(
   

In [ ]:
import numpy as np
import torch
import open3d as o3d

# --- Load and prepare data ---
sample_path = "preprocessed_data/sample_200.npz"
data = np.load(sample_path)
points = data["points"]  # [8192, 6]
xyz = points[:, :3]      # just xyz for Open3D visualization

# --- Prepare tensor for inference ---
points_tensor = torch.from_numpy(points).unsqueeze(0).float().to(device)  # [1, 8192, 6]
points_tensor = points_tensor.transpose(1, 2)  # [1, 6, 8192] for PointNet++

# --- Inference ---
model.eval()
with torch.no_grad():
    pred_logits, _ = model(points_tensor)  # [1, 8192, 2]
    pred_labels = pred_logits.argmax(dim=2).squeeze().cpu().numpy()  # [8192]

# --- Map labels to colors ---
colors = np.zeros_like(xyz)
colors[pred_labels == 0] = [0.6, 0.6, 0.6]  # gray (background)
colors[pred_labels == 1] = [1.0, 0.0, 0.0]  # red (screw)

# --- Visualize in Open3D ---
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_geometries([pcd])


[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
